In [1]:
import pandas as pd
from tqdm import tqdm
import random
import itertools
import torch.nn as nn
import torch
from sklearn.neighbors import KernelDensity
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import norm
import plotly.figure_factory as ff

random.seed(42)
cos = nn.CosineSimilarity(dim=0, eps=1e-6)

In [2]:
def normalize_lists(l1, l2):
    n = len(l1)

    l2 = l2.sample(n=n, random_state=42)

    return l1, l2

In [23]:
def sim(same, diff, cl):
    same = same.reset_index(drop=True)
    diff = diff.reset_index(drop=True)

    same_pairs = list(itertools.combinations(same.index.to_list(), 2))
    diff_pairs = list(itertools.combinations(diff.index.to_list(), 2))

    n = min(2000, len(same_pairs))

    same_pairs = random.sample(same_pairs, n)
    diff_pairs = random.sample(diff_pairs, n)

    root = "/pless_nfs/home/mdt_/Hotels50-FeatureComputation/"

    same_sim, diff_sim = [], []

    for item in zip(same_pairs, diff_pairs):
        s1 = root + same.iloc[item[0][0]]["Path"] + "/indiv_features1.pt"
        s2 = root + same.iloc[item[0][1]]["Path"] + "/indiv_features1.pt"
        
        d1 = root + diff.iloc[item[1][0]]["Path"] + "/indiv_features1.pt"
        d2 = root + diff.iloc[item[1][1]]["Path"] + "/indiv_features1.pt"

        same_sim.append( float(cos(torch.load(s1)[same.iloc[item[0][0]]["Idx"]], torch.load(s2)[same.iloc[item[0][1]]["Idx"]])) )
        diff_sim.append( float(cos(torch.load(d1)[diff.iloc[item[1][0]]["Idx"]], torch.load(d2)[diff.iloc[item[1][1]]["Idx"]])) )
        

    return same_sim, diff_sim



In [4]:
csv = pd.read_csv("/pless_nfs/home/mdt_/Hotels50-FeatureComputation/data/train/features.csv", names=["Idx", "Class", "Path"])
csv["Instance"] = [row.split("/")[3] for row in csv["Path"]]

In [5]:
instances = csv["Instance"].unique()
classes = csv["Class"].unique()

In [6]:
ade_classes = pd.read_csv("/pless_nfs/home/mdt_/Hotels50-FeatureComputation/data/features_150.csv")
"tewt" + ade_classes.loc[ade_classes["Idx"] == 1]["Name"]

0    tewtwall
Name: Name, dtype: object

In [24]:
for cl in tqdm(classes):
    total_same, total_diff = [], []

    set(csv.loc[csv["Class"] == cl]["Instance"].to_list())

    for instance in tqdm(instances):
    
        same, diff = csv.loc[csv["Instance"] == instance], csv.loc[csv["Instance"] != instance]
        same_sim, diff_sim = sim(same, diff, cl)
        total_same.extend(same)
        total_diff.extend(diff)
        # print("SAME: " + str(same_sim), "DIFF: " + str(diff_sim), sep="\n")
        # kde_same = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(np.array(same_sim).reshape(-1, 1))
        # kde_diff = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(np.array(diff_sim).reshape(-1, 1))

        # x = np.array(same_sim).reshape(-1, 1)
        # x_d = np.linspace(-1, 2, 1000)
        # logprob = kde_same.score_samples(x_d[:, None])
        # plt.fill_between(x_d, np.exp(logprob), alpha=0.5)
        # plt.plot(x, np.full_like(x, -0.01), '|k', markeredgewidth=1)
        # plt.ylim(0, 2)


    group_labels = ['Same', 'Diff']
    colors = ['#333F44', '#37AA9C']

    # Create distplot with curve_type set to 'normal'
    fig = ff.create_distplot([total_same, total_diff], group_labels, show_hist=False, colors=colors, show_rug=False)

    # Add title
    fig.update_layout(title_text='Feature: {}- Similarities Same and Diff Inst of Hotels'.format(feature))
    fig.show()

    feature = ade_classes.loc[ade_classes["Idx"] == cl]["Name"]
    path = "data/figures/train/{}/".format(feature)
    os.makedirs(path)

    fig.write_image(path + "same-diff-instances.png")

            


  0%|          | 0/7572 [00:00<?, ?it/s]

In [22]:
len(set(csv.loc[csv["Class"] == 6]["Instance"].to_list()))

1988